<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/rust01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rust をインストールする

In [18]:
# Rust をインストールする
%%capture
!apt install rustc;

In [19]:
# インストールの確認
!rustc --version

rustc 1.43.0


In [20]:
# プログラムを書く
%%writefile main.rs
fn main() {
    println!("Hello, world!");
}

Overwriting main.rs


In [21]:
# コンパイルして実行
!rustc main.rs
!./main

Hello, world!


In [22]:
# 実験
# 名前を買えてみる
%%writefile hoge.rs
fn main() {
    println!("hoge, world!");
}

Overwriting hoge.rs


In [23]:
# 実験
# 1行でコンパイルして、実行
!rustc hoge.rs; ./hoge

hoge, world!
